# Toy Simulacra 1.1

This is the first in a series of four notebooks designed to build a simplified version of Simulacra [1]. While the original Simulacra encompasses a broader range of components facilitating comprehensive simulations, this tutorial series aims to familiarize the audience with the implementation of generative agents and the application of Language Learning Models (LLMs) in sociological simulations.

For additional insights into how this tutorial is organized and some overarching considerations in constructing simulations with LLMs, I invite you to read a retrospective on my [blog](https://www.pgupta.info/blog).

[[[1] Park et al. 2023 Generative Agents: Interactive Simulacra of Human Behavior](https://arxiv.org/abs/2304.03442)

Note: Parts of this tutorial series have been refined using LLMs, guided by prompts like "Proofread and correct --".

## Objectives

In this part of our tutorial, we’ll focus on understanding how to interact with the simulated environment, setting aside the complexities of agent-specific details. To facilitate this, we'll introduce a dummy agent for the purposes of demonstration.


## Setup

**Important Note:** For those who prefer running the code via the command line rather than navigating through notebook cells—which can become tedious due to the volume of code in subsequent notebooks—I've made the entire codebase available [here](https://github.com/pg2455/toy-simulacra).

### Requirements:
- **OpenAI Library**: Install using the command `pip install openai`.
- **GPT Keys**: Required for sections 1.3 and 1.4 to interact with LLMs.

**Additional Note:** While this tutorial is crafted around the use of GPT, it's adaptable to other LLMs. Should you choose to use an alternative LLM, simply adjust the `PROMPT_FN` in notebooks 1.3 and 1.4. Anticipate approximately 1000 calls to the LLMs for a simulation spanning 1.5 simulation days.

### Getting Started:

1. Clone the original repository to access the necessary files:
   ```bash
   git clone https://github.com/joonspk-research/generative_agents.git
   ```
2. Copy the relevant files into your current working directory:
   ```bash
   cp ./generative_agents/reverie/backend_server/maze.py .
   cp ./generative_agents/reverie/backend_server/global_methods.py .
   cp ./generative_agents/reverie/backend_server/utils.py .
   ```
Alternatively, execute the code block provided to automate this process.

In [ ]:
!git clone https://github.com/joonspk-research/generative_agents.git
!cp ./generative_agents/reverie/backend_server/maze.py . 
!cp ./generative_agents/reverie/backend_server/global_methods.py . 
!cp ./generative_agents/reverie/backend_server/utils.py . 

In [1]:
import json
import os
import time
import random
from pathlib import Path
from openai import AzureOpenAI
from datetime import datetime, timedelta
from maze import Maze

BASE_SIM_FOLDER = Path("./generative_agents/environment/frontend_server/storage/base_the_ville_isabella_maria_klaus/").resolve()
PERSONAS_FOLDER = BASE_SIM_FOLDER / "personas"

## Environment: The Maze

The environment is encapsulated within the `Maze` class, found in `maze.py`. This class is designed to be straightforward, offering functionalities to:
- Interact with the environment,
- Store the current state of the environment, and
- Provide access to various aspects of itself.

Utilizing these functions, we'll explore how to manage interactions between our dummy agent and the environment.

In [2]:
maze = Maze("the Ville")

print(f"Height:{maze.maze_height}\tWidth{maze.maze_width}")

Height:100	Width140


In [3]:
# Returns the nearby tiles (vision_r defines the radius around the coordinate)
maze.get_nearby_tiles([72,14], vision_r=1) 

[(71, 13),
 (71, 14),
 (71, 15),
 (72, 13),
 (72, 14),
 (72, 15),
 (73, 13),
 (73, 14),
 (73, 15)]

Each tile has a string address in addition to the numerical coordinates. 
This string address is created as such: World name: Sector name: Arena name: Object name

World name is the name of the environment. This will be constant across all the tiles. 
Sector name is the name of the block for example school or cafe
Arena name is the name of the small sub sections for example rooom in a cafe, etc. 
Object name is the name of the object on that tile (if any) such as desk, table etc. 



In [4]:
# Determine the string address of the tile
# World: Sector (e.g., house, cafe): Arena (e.g., designated area within sector): object (An actual object, e.g., bed, table, etc.)
maze.get_tile_path([72, 14], level='object')

"the Ville:Isabella Rodriguez's apartment:main room:bed"

## Events

Events represent perceptions that are shared between agents and the environment, as well as among the agents themselves. They can be as varied as "The piano is occupied" or "Maria is chatting with Isabella." These shared perceptions will be elaborated upon through `ConceptNode` in the next notebook, where it will become apparent that events encapsulate more than just textual descriptions.

Defined by a structure (subject, predicate, object, description), events also possess additional attributes such as the time of creation, expiration, and embeddings. These embeddings play a crucial role in the simulation, particularly in determining an event’s relevance to specific queries, for instance, by calculating the cosine similarity with a query’s embedding. This process will be detailed in Notebook 1.3.

While the original Simulacra code distinguishes between chats, thoughts, and environmental events, for ease of understanding, we have consolidated these distinctions into a singular concept of 'events'. This approach simplifies the code and will be explored further in the forthcoming notebook.

Lastly, when an event is associated with a specific location in the environment, each corresponding tile records the event. Thus, when an agent perceives a tile within its perception radius, the event is registered in the agent's short-term memory, seamlessly integrating environmental dynamics with individual agent experiences.


In [5]:
# Get a more detailed information about a specific tile
# Note: collision defines whether there is a wall or an object that can block person's path
# Note: events is a set that determines all events on that tile. By default, tile_path is the event. See below for events.
maze.access_tile([72, 14])

{'world': 'the Ville',
 'sector': "Isabella Rodriguez's apartment",
 'arena': 'main room',
 'game_object': 'bed',
 'spawning_location': 'sp-A',
 'collision': False,
 'events': {("the Ville:Isabella Rodriguez's apartment:main room:bed",
   None,
   None,
   None)}}

## Persona

Following our exploration of the Maze, we now turn our attention to the Persona class, which encapsulates the essence of a generative agent. This class is integral to crafting "believably human" behaviors by enabling agents to perform actions, exhibit behaviors, and make plans that are consistent with their characters. In this discussion, we will introduce dummy versions of the various memory structures utilized by Simulacra, rather than delving into their intricate details:

- **SpatialMemoryTree**: A structure that represents the agent's knowledge of the environment.
- **AssociativeMemory**: A repository for the agent's thoughts, conversations, and events, encapsulating their experiences.
- **Scratch (Short-Term Memory)**: A class for storing transient information, such as daily plans, current actions, and identity details like agendas and intended paths.

In the subsequent notebook, we'll delve deeper into these constituent classes, further illustrating its role in simulating human-like behavior.

In [12]:
class Scratch:
    def __init__(self, fname):
        self.name = "dummy"
        self.curr_time = None
        self.curr_tile = [72, 14]

    def get_curr_event_and_desc(self):
        return (self.name, None, None, None)

class AssociativeMemory:
    def __init__(self, fname):
        pass

class SpatialMemoryTree:
    def __init__(self, fnam):
        pass

class Persona:
    def __init__(self, name, folder_mem, curr_time, initiate_plan=True):
        self.name = name
        self.s_mem = None 
        self.a_mem = None
        self.scratch = Scratch(f"{folder_mem}/bootstrap_memory/scratch.json")
        self.s_mem = SpatialMemoryTree(f"{folder_mem}/bootstrap_memory/scratch.json")
        self.a_mem = AssociativeMemory(f"{folder_mem}/bootstrap_memory/scratch.json")
        self.scratch.curr_time = curr_time
        
        if initiate_plan:
            self.generate_day_plan()

    def generate_day_plan(self):
        pass

    def perceive_and_retrieve_and_focus(self):
        pass

    def advance_one_step(self, maze, personas, curr_time):
        x = self.get_curr_tile()
        # Randomly generating new tile
        new_tile = [x[0]+random.randint(-5, 5), x[1]+random.randint(-5, 5)]
        return new_tile

    def get_curr_tile(self):
        return self.scratch.curr_tile

    def move(self, new_tile):
        self.scratch.curr_tile = new_tile
        

### Simulation loop

Finally, we need a simulation clock that can keep itself turning every regular interval. We chose this interval based on our required precision. For this tutorial, I have chosen it to be 10 minutes per simulation iteration. Thus, each simulation iteration is akin to 10 minutes real time. 

This is done in the main loop, where we keep advancing the simulation clock and between each tick, we check on each agent and advance them one by one - perceiving the world around them, enchancing their memories, taking actions, adapting their plans along the way. 

Note we iterate through the agents twice. This is so that within the same iteration, perception should be symmetric. If we move the agent in the first iteration, that agent might not be visible to the other agents in the same iteration (leading to asymmetric perception). 


In [13]:
maze = Maze("the Ville")
curr_time = sim_start_time = datetime(2024, 2, 13, 0, 0, 0) # Start at midnight
seconds_per_step = 10 * 60 # 10 minutes
n_steps = 5

personas = []
for persona_folder in PERSONAS_FOLDER.iterdir():
    personas.append(Persona(persona_folder.name, persona_folder, curr_time, initiate_plan=True))

step = 0
movements = {}
while step < n_steps:

    for persona in personas:
        curr_tile = persona.get_curr_tile()
        new_tile = persona.advance_one_step(maze, personas, curr_time)
        movements[persona.name] = new_tile

    for persona in personas:
        new_tile = movements[persona.name]
        if new_tile:
            maze.remove_subject_events_from_tile(persona.name, curr_tile)
            maze.add_event_from_tile(persona.scratch.get_curr_event_and_desc(), new_tile)
            persona.move(new_tile)
        print(curr_time.strftime("%H:%M"), persona.name, persona.scratch.curr_tile)
    
    step += 1
    curr_time = sim_start_time + timedelta(seconds=seconds_per_step*step)

00:00 Klaus Mueller [72, 16]
00:00 Isabella Rodriguez [67, 14]
00:00 Maria Lopez [70, 9]
00:10 Klaus Mueller [77, 21]
00:10 Isabella Rodriguez [72, 17]
00:10 Maria Lopez [70, 12]
00:20 Klaus Mueller [74, 26]
00:20 Isabella Rodriguez [77, 18]
00:20 Maria Lopez [68, 11]
00:30 Klaus Mueller [75, 29]
00:30 Isabella Rodriguez [82, 13]
00:30 Maria Lopez [70, 7]
00:40 Klaus Mueller [79, 29]
00:40 Isabella Rodriguez [80, 8]
00:40 Maria Lopez [66, 8]


With the foundational elements of our simulation now established, we're ready to delve deeper into the memory structure in our upcoming notebook.